In [1]:
%matplotlib inline

In [2]:
from ds4400_final_project.dataset.constants import DATASET_FOLDER
from pathlib import Path
from typing import Tuple, Dict
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np

In [3]:
def load_data_from_file(csv_filename: str) -> Tuple[np.array, np.array, Dict[int, str], Dict[str, int]]:
	""" Load the CSV file from the dataset folder. """
	file = str(Path(DATASET_FOLDER) / csv_filename)
	features_list = np.genfromtxt(file, dtype=None, encoding=None, delimiter=",", skip_header=1, usecols=range(2, 60))
	features = np.array([list(x) for x in features_list])

	# Create a mapping between a numeric value and genre
	index_genre_map = {i: genre for i, genre in enumerate(np.unique(features[:,-1]))}
	genre_index_map = {value: key for key, value in index_genre_map.items()}

	# split the inputs and their labels
	x = normalize(features[:,:57])
	# y = normalize(np.array([genre_index_map[genre] for genre in features[:,-1]]).reshape(-1, 1))

	# y = np.array([make_one_hot(genre_index_map[genre]) for genre in features[:,-1]])
	y = np.array([genre_index_map[genre] for genre in features[:,-1]])

	return x, y, index_genre_map, genre_index_map

In [4]:
# import the data from the 3 seconds and 30 seconds features CSV
X_3, y_3, index_genre_map_3, genre_index_map_3 = load_data_from_file("features_3_sec.csv")
X_30, y_30, index_genre_map_30, genre_index_map_30 = load_data_from_file("features_30_sec.csv")

In [5]:
# split all the data into training and testing sets
TEST_SIZE = 0.33
RANDOM_STATE = 42

x_3_train, x_3_test, y_3_train, y_3_test = train_test_split(X_3, y_3, test_size=TEST_SIZE, random_state=RANDOM_STATE)
x_30_train, x_30_test, y_30_train, y_30_test = train_test_split(X_30, y_30, test_size=TEST_SIZE, random_state=RANDOM_STATE)
x_3_train, x_3_test, x_30_train, x_30_test = map(torch.FloatTensor, [x_3_train, x_3_test, x_30_train, x_30_test])
y_3_train, y_3_test, y_30_train, y_30_test = map(torch.LongTensor, [y_3_train, y_3_test, y_30_train, y_30_test])

In [6]:
class ShallowModel(nn.Module):

    def __init__(self, num_hidden: int):
        super().__init__()
        self.hidden = nn.Linear(57, num_hidden)
        self.output = nn.Linear(num_hidden, 10)
        self.softmax = nn.LogSoftmax(dim=1)
        

    def forward(self, x):
        x = torch.relu(self.hidden(x))
        x = self.output(x)
        return self.softmax(x)

In [7]:
class DeepModelTwoHiddenLayer(nn.Module):

    def __init__(self, num_hidden_nodes_1: int, num_hidden_nodes_2: int):
        super().__init__()
        self.hidden1 = nn.Linear(57, num_hidden_nodes_1)
        self.hidden2 = nn.Linear(num_hidden_nodes_1, num_hidden_nodes_2)
        self.output = nn.Linear(num_hidden_nodes_2, 10)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        x = self.output(x)
        return self.softmax(x)

In [8]:
class GTZANDataset(Dataset):

    def __init__(self, x_data, y_data):
        self._x_data = x_data
        self._y_data = y_data

    def __len__(self):
        return len(self._x_data)
        
    def __getitem__(self, idx):
        return [self._x_data[idx], self._y_data[idx]]

In [359]:
def train_model(model, loss_func, x_train, y_train, epochs: int, lr: float, l2_reg: float, batch_size: int, verbose = False):
	dataset = GTZANDataset(x_train, y_train)
	dataloader = DataLoader(dataset, batch_size=batch_size)

	optimizer = torch.optim.SGD(params=model.parameters(), lr=lr, weight_decay=l2_reg)

	# set the model to training mode
	model.train()

	# save average losses every epoch
	losses = []

	# optimize the model
	for epoch in range(epochs):

		print(f"\nEpoch {epoch+1}\n-------------------------------") if verbose else None

		losses_each_batch = []

		for batch, (x, y) in enumerate(dataloader):

			y_pred = model(x)

			loss = loss_func(y_pred, y)

			# backpropogation
			optimizer.zero_grad()
			loss.backward()
			optimizer.step()

			# save the loss for this batch
			losses_each_batch.append(loss.item())

			if verbose and batch % (batch_size // 10) == 0:
				avg_running_loss = sum(losses_each_batch) / len(losses_each_batch)
				current = batch * len(x)
				print(f"Epoch [{epoch:03}] Batch [{batch:03}] avg_running_loss: {avg_running_loss:>7f} [{current:>5d}/{len(dataset):>5d}]")

		# average all the batch losses for this epoch and save it
		avg_loss_epoch = sum(losses_each_batch) / len(losses_each_batch)
		losses.append(avg_loss_epoch)

		if verbose:
			print(f"Epoch [{epoch:03}]: average loss: {avg_loss_epoch:>7f}")

	# set the model to evaluation mode
	model.eval()
	return model, losses

In [379]:
def run_training_experiments(experiment_configs, x_train, y_train):

    models = {}

    for model_id, config in experiment_configs.items():
        model_class = config["model_class"]
        model_kwargs = config["model_kwargs"] if "model_kwargs" in config.keys() else {} 
        model_instance = model_class(**model_kwargs)
        loss_func_class = config["loss_func_class"]
        loss_func_kwargs = config["loss_func_kwargs"] if "loss_func_kwargs" in config.keys() else {}
        loss_func = loss_func_class(**loss_func_kwargs)

        print(f"TRAINING MODEL: {model_id}")

        output_model, losses = train_model(
            model=model_instance,
            loss_func=loss_func,
            x_train=x_train,
            y_train=y_train,
            epochs=config["epoch"],
            lr=config["lr"],
            l2_reg=config["l2_reg"],
            batch_size=config["batch_size"],
            verbose=config["verbose"]
        )

        models[model_id] = {"model": output_model, "losses": losses}

    return models


In [380]:
EPOCHS = 2000
LEARNING_RATE = 0.01
L2_REGULARIZATION = 0.001
BATCH_SIZE = 128

experiments = {
    "Shallow Model 32 Hidden Nodes": {
        "model_class": ShallowModel,
        "model_kwargs": {
            "num_hidden": 32
        },
        "loss_func_class": nn.NLLLoss,
        "epoch": 2000,
        "lr": LEARNING_RATE,
        "l2_reg": L2_REGULARIZATION,
        "batch_size": BATCH_SIZE,
        "verbose": True
    },
    "Shallow Model 64 Hidden Nodes": {
        "model_class": ShallowModel,
        "model_kwargs": {
            "num_hidden": 64
        },
        "loss_func_class": nn.NLLLoss,
        "epoch": 2000,
        "lr": LEARNING_RATE,
        "l2_reg": L2_REGULARIZATION,
        "batch_size": BATCH_SIZE,
        "verbose": True
    },
    "Shallow Model 128 Hidden Nodes": {
        "model_class": ShallowModel,
        "model_kwargs": {
            "num_hidden": 128
        },
        "loss_func_class": nn.NLLLoss,
        "epoch": 2000,
        "lr": LEARNING_RATE,
        "l2_reg": L2_REGULARIZATION,
        "batch_size": BATCH_SIZE,
        "verbose": True
    },
    "Deep Model 2 Hidden Layers 64 - 32 nodes": {
        "model_class": DeepModelTwoHiddenLayer,
        "model_kwargs": {
            "num_hidden_nodes_1": 64,
            "num_hidden_nodes_2": 32
        },
        "loss_func_class": nn.NLLLoss,
        "epoch": 2000,
        "lr": LEARNING_RATE,
        "l2_reg": L2_REGULARIZATION,
        "batch_size": BATCH_SIZE,
        "verbose": True
    },
    "Deep Model 2 Hidden Layers 128 - 64 nodes": {
        "model_class": DeepModelTwoHiddenLayer,
        "model_kwargs": {
            "num_hidden_nodes_1": 128,
            "num_hidden_nodes_2": 64
        },
        "loss_func_class": nn.NLLLoss,
        "epoch": 2000,
        "lr": LEARNING_RATE,
        "l2_reg": L2_REGULARIZATION,
        "batch_size": BATCH_SIZE,
        "verbose": True
    },
}

In [381]:
three_sec_training_results = run_training_experiments(experiment_configs=experiments, x_train=x_3_train, y_train=y_3_train)

TRAINING MODEL: Shallow Model 32 Hidden Nodes

Epoch 1
-------------------------------
Batch [000]: loss: 2.321122  [    0/ 6693]

Epoch 2
-------------------------------
Batch [000]: loss: 2.318990  [    0/ 6693]

Epoch 3
-------------------------------
Batch [000]: loss: 2.317155  [    0/ 6693]

Epoch 4
-------------------------------
Batch [000]: loss: 2.315562  [    0/ 6693]

Epoch 5
-------------------------------
Batch [000]: loss: 2.314153  [    0/ 6693]

Epoch 6
-------------------------------
Batch [000]: loss: 2.312911  [    0/ 6693]

Epoch 7
-------------------------------
Batch [000]: loss: 2.311828  [    0/ 6693]

Epoch 8
-------------------------------
Batch [000]: loss: 2.310866  [    0/ 6693]

Epoch 9
-------------------------------
Batch [000]: loss: 2.310010  [    0/ 6693]

Epoch 10
-------------------------------
Batch [000]: loss: 2.309241  [    0/ 6693]

Epoch 11
-------------------------------
Batch [000]: loss: 2.308543  [    0/ 6693]

Epoch 12
------------------

KeyboardInterrupt: 

In [ ]:
def plot_loss(outputs):

    legend = []
    for model_id, output in outputs.items():
        legend.append(model_id)
        losses = output["losses"]
        plt.plot(range(losses), losses)
    plt.legend(legend, loc='upper left')
    plt.title("Training loss")
    plt.show()

In [ ]:
plot_loss(outputs=three_sec_training_results)

In [ ]:
model.eval()
y_pred = model(x_3_test)
after_train = loss_function(y_pred.squeeze(), y_3_test) 
print('Test loss after Training' , after_train.item())

Test loss after Training 2.3030574321746826
